In [1]:
import os
import logging
import concurrent.futures
from typing import List, Dict, Any, Callable, Union, Optional
from pydantic import BaseModel
from prompts import ANTHROPIC_DEFAULT_PROMPT
from haystack import Pipeline, Document, component
from haystack.utils import Secret
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore
from haystack.components.generators import OpenAIGenerator as BaseOpenAIGenerator
from haystack.components.builders import PromptBuilder
from haystack.dataclasses import ChatMessage, StreamingChunk
from openai import OpenAI, Stream
from openai.types.chat import ChatCompletion, ChatCompletionChunk
from haystack.components.generators.openai_utils import _convert_message_to_openai_format
import concurrent.futures
from typing import List, Dict
from tqdm import tqdm
from haystack import component
from haystack.dataclasses import Document
from anthropic import Anthropic
from dotenv import load_dotenv

e:\programming\HUA-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()
# Logging config
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
import json
import os

def enrich_data(data_path, fulltext_path):
    """
    Enriches the JSON data with fulltext content based on matching 'invnr'.
    Adds 'Keuze': 'JA' if the key does not exist

    Args:
        data_path (str): Path to the JSON data file.
        fulltext_path (str): Path to the JSON fulltext data file.

    Returns:
        list: The enriched list of dictionaries.
    """
    with open(data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    with open(fulltext_path, 'r', encoding='utf-8') as f:
        fulltext_data = json.load(f)

    for item in data:
        if 'Keuze' not in item:
           item['Keuze'] = 'JA'
        invnr_to_find = str(item.get('invnr'))
        if invnr_to_find is None:
            continue

        for fulltext_entry in fulltext_data:
            if str(fulltext_entry.get('invnr')) == invnr_to_find:
                item['fulltext'] = fulltext_entry.get('content')
                break  # stop searching once we found a match

    return data

if __name__ == '__main__':
    # Replace with your actual file paths
    data_file_path = '../data/prototyping/hua.json' #<---- set path here
    fulltext_file_path = '../data/prototyping/HUA-fulltext.json' #<---- set path here

    # Make sure the files exist before processing
    if not os.path.exists(data_file_path):
        print(f"Error: Data file not found at '{data_file_path}'.")
    elif not os.path.exists(fulltext_file_path):
        print(f"Error: Fulltext data file not found at '{fulltext_file_path}'.")
    else:
        enriched_data = enrich_data(data_file_path, fulltext_file_path)
        
        # Output the enriched JSON to a file or stdout for inspection
        with open('enriched_data.json', 'w', encoding='utf-8') as outfile:
            json.dump(enriched_data, outfile, indent=2, ensure_ascii=False)
        

In [4]:
import json

def filter_by_keuze_and_fulltext(file_path):
    """
    Loads JSON data, filters entries where 'Keuze' is 'ja' (case-insensitive)
    AND 'fulltext' is not None, and returns a new list with only the
    matching entries.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        list: A list of dictionaries where 'Keuze' was 'ja' (case-insensitive)
              and 'fulltext' was not None.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    filtered_list = [
        item for item in data
        if item.get('Keuze', '').lower() == 'ja' and item.get('fulltext') is not None
    ]

    return filtered_list


# Replace with your actual file path
file_path = 'enriched_data.json'

filtered_data = filter_by_keuze_and_fulltext(file_path)



In [5]:
docs = []
for data in filtered_data:
    doc = Document(content=data['fulltext'], meta={key: value for key, value in data.items() if key != 'fulltext'})
    docs.append(doc)

In [6]:
def get_openai_client() -> OpenAI:
    return OpenAI()

def translate(text: str) -> str:
    client = get_openai_client()
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "Je bent een expert in het vertalen van teksten van oud-nederlands naar Nederland. Je krijgt een collectie aan HTR teksten die je meot vertalen naar het hedendaags Nederlands. Het kan zijn dat er stukken tekst bij zitten die niet volledig te lezen zijn, of niet volledig te bevatten zijn. In dat geval kan je het laten staan zonder bewerking. Geef ENKEL de vertaling terug, geen 'alsjeblieft', 'hier', etc. "},
            {"role": "system", "content": "Als er veel typfouten in zitten zeg je hier niks van. Laat de tekst onbewerkt en probeer de tekst die wel leesbaar is zo goed mogelijk te vertalen. Vervang onleesbaar met ___(onleesbaar)___. Vervang verder NOOIT een stuk tekst met een placeholder."},
            {"role": "system", "content": "Als je een input krijgt, vertaal je deze en zeg je verder niks anders. Behoud de tekst zoals het is, vervang geen stukken, geen interpretaties, geen opvulling, geen [...], (hetzelfde) of andere opvullende tekens behalve bij onleesbare tekst. De tekst blijft zo origineel mogelijk"},
            {"role": "user", "content": f"{text}"},
        ],
    )
    return response.choices[0].message.content

@component
class DocumentTranslator:
    @component.output_types(docs=List[Document])
    def run(self, documents: List[Document]) -> Dict[str, List[Document]]:
        translated_documents = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            for document in documents:
                current_text = document.content
                future = executor.submit(translate, current_text)
                futures.append((document, future))

            for document, future in tqdm(futures):
                translated = future.result()
                document.content = translated
                translated_documents.append(document)
        return {"docs": translated_documents}




In [7]:
from pydantic import Field
import json
import os
from typing import List, Dict, Any, Optional, Union
from openai import OpenAI
from openai.types.chat import ChatCompletion
from pydantic import BaseModel, Field
from haystack import component
from haystack.dataclasses import Document
from haystack.core.pipeline import Pipeline
from haystack.utils import Secret, deserialize_secrets_inplace
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators.openai_utils import _convert_message_to_openai_format

DEFAULT_PROMPT = ANTHROPIC_DEFAULT_PROMPT


# DEFAULT_PROMPT = """
# **Instructies voor het LLM:**

# **Invoer:**
# - URL van het item
# - Inhoud van het item (dit kan een brief, kaart, foto, artikel of een ander type document zijn)
# - Titel van het item (als beschikbaar)
# - Beschrijving van het item (als beschikbaar)

# **Doel:**  
# Maak, op basis van de aangeleverde informatie, een uitgebreide metadatabeschrijving in het Nederlands.

# ---

# ### Stap 1: Bepaal het type item (AssetType)

# Analyseer de inhoud van het item en bepaal welk type het is. Mogelijke types zijn:

# - **Brief** (Letter)  
# - **Kaart** (Map)  
# - **Foto Gebouw** (Photo of a Building)  
# - **Foto Interieur** (Photo of an Interior)  
# - **Foto Mensen** (Photo of People)  
# - **Foto Album** (Photo Album)  
# - **Boek** (Book)  
# - **Artikel** (Article)  
# - **Akte** (Official Document)  

# ---

# ### Stap 2: Haal gedetailleerde, item-specifieke metadata op

# Zodra het type is bepaald, verzamel zoveel mogelijk relevante metadata die specifiek is voor dat type.

# **Voor Brieven (Brief):**  
# - **Sender:** Naam van de afzender.  
# - **Recipient:** Naam van de ontvanger.  
# - **DateSent:** Datum waarop de brief is verstuurd.  
# - **LetterType:** Type brief (bijv. Persoonlijk, Officieel).  
# - **ContentSummary:** Korte samenvatting van de inhoud van de brief.

# **Voor Kaarten (Kaart):**  
# - **Scale:** Schaal van de kaart (bijv. 1:5000).  
# - **MapFeatures:** Opvallende kenmerken, zoals gebouwen, grenzen, rivieren.  
# - **DateCreated:** Datum waarop de kaart is gemaakt.  
# - **LocationCovered:** Gebieden of plaatsen die op de kaart worden weergegeven.

# **Voor Foto’s (Foto):**  
# - **Photographer:** Naam van de fotograaf.  
# - **DateTaken:** Datum waarop de foto is genomen.  
# - **Event:** Gebeurtenis die op de foto is vastgelegd.  
# - **PeopleInPhoto:** Namen van personen op de foto.  
# - **Description:** Gedetailleerde beschrijving van de foto (bijv. kleding, gebouwen, activiteiten).

# **Voor Artikelen (Artikel):**  
# - **Author:** Naam van de auteur.  
# - **PublicationDate:** Datum van publicatie.  
# - **Source:** Naam van het tijdschrift of de bron.  
# - **ArticleTopic:** Onderwerp of thema van het artikel.

# ---

# ### Stap 3: Genereer de overkoepelende (Master) Metadata

# Gebruik alle invoer (bestaande metadata, URL) en de item-specifieke metadata om de volgende overkoepelende metadata-velden aan te maken:

# | Veld                | Beschrijving                                                                                                  |
# |---------------------|--------------------------------------------------------------------------------------------------------------|
# | AssetID             | Een unieke identificatiecode voor het item.                                                                   |
# | Title               | De officiële naam of titel van het item.                                                                      |
# | Description         | Een korte samenvatting of beschrijving van de inhoud.                                                         |
# | PrimaryQuestion     | De belangrijkste vraag die dit item beantwoordt.                                                              |
# | PrimaryTheme        | Het hoofdthema waar dit item bij hoort.                                                                       |
# | SecondaryThemes     | Specifieke subthema's die het hoofdthema verfijnen.                                                          |
# | Entities            | Belangrijke personen, plaatsen of objecten die aan het item zijn gekoppeld.                                    |
# | EntityRelationships | Beschrijvingen van relaties tussen entiteiten.                                                               |
# | TimePeriod          | De historische periode of datum die met het item wordt geassocieerd.                                          |
# | Location            | De geografische context die bij het item hoort.                                                              |
# | AssetType           | Het type item (bijv. Brief, Foto, Kaart).                                                                     |
# | StorylineDimension  | Dominante narratieve structuur (Chronologisch, Entiteitgericht, Emotiegedreven).                              |
# | NarrativeFocus      | Hoe het item bijdraagt aan het verhaal of narratief.                                                         |
# | URL                 | Link naar het item voor ophalen en presentatie.                                                              |
# | Keywords            | Extra zoekwoorden of tags voor betere zoekresultaten.                                                         |
# | ExplorationTags     | Gerelateerde thema's of onderwerpen voor verdere verkenning.                                                  |
# | FollowUpQuestionTags| Tags voor het genereren van vervolgvragen.                                                                    |
# | FullText            | De volledige tekst van het item (indien van toepassing).                                                      |
# | Summary             | Een korte samenvatting van de volledige tekst.                                                               |

# ---

# ### Stap 4: Maak de gecombineerde JSON-uitvoer

# Genereer een JSON-structuur die zowel de item-specifieke metadata als de overkoepelende metadata bevat. Zorg ervoor dat alles in het Nederlands geformuleerd is.

# }
# Taalvereisten:
# Alle uitvoer (inclusief titels, beschrijvingen, metadatavelden, volledige tekst, samenvattingen en item-specifieke metadata) dient in het Nederlands te zijn.
# Gebruik helder en correct Nederlands, passend bij archiefmateriaal in Nederland.
# Zorg voor correcte Nederlandse spelling, grammatica en woordkeuze.
# Houd rekening met culturele en historische contexten als dit relevant is voor het materiaal.
# Formatteerrichtlijnen:
# Als het item namen, plaatsen of terminologie uit de Nederlandse geschiedenis bevat, bewaar deze dan in het Nederlands.
# Voor historische documenten gebruik bij voorkeur taal die past bij de tijdsperiode (bijv. 19e-eeuws woordgebruik).
# Voor moderne omschrijvingen en samenvattingen gebruik hedendaags, algemeen begrijpelijk Nederlands.

# {{document}}

# """



In [8]:
from pydantic import ValidationError


@component
class BaseOpenAIGenerator(object):
    """
    A component that uses OpenAI's models to generate text.

    If you want to use a model that supports chat-like interactions, use the `OpenAIChatGenerator` instead.

    """

    def __init__(
        self,
        api_key: Secret = Secret.from_env_var("OPENAI_API_KEY"),
        model: str = "gpt-4o",
        generation_kwargs: Optional[Dict[str, Any]] = None,
        system_prompt: Optional[str] = None,
    ):
        """
        :param api_key: OpenAI API key.
        :param model: The name of the model to use.
        :param generation_kwargs: Additional kwargs for the OpenAI completion endpoint.
             See the OpenAI API reference for a full list of available parameters:
             https://platform.openai.com/docs/api-reference/completions/create
        :param system_prompt: The system prompt to be used by the model.
            When used in a conversational scenario, this prompt is sent to the model before the user query to
            guide it towards a desired behavior. If not set, the model will only receive user messages.
        """
        self.api_key = api_key
        self.model = model
        self.generation_kwargs = generation_kwargs or {}
        self.system_prompt = system_prompt
        self.client = OpenAI(api_key=self.api_key.resolve_value())

    def _check_finish_reason(self, response: ChatMessage):
        if response.meta.get("finish_reason") == "length":
            raise ValueError(
                f"""The completion for the current document ended due to the 'length' of the response.
                      Consider increasing the max_tokens parameter in the generation_kwargs or the size of the document.
                   """
            )

    @component.output_types(replies=List[str], meta=List[Dict[str, Any]])
    def run(
        self,
        prompt: str,
        generation_kwargs: Optional[Dict[str, Any]] = None
    ):
        generation_kwargs = {**self.generation_kwargs, **(generation_kwargs or {})}

        message = ChatMessage.from_user(prompt)
        if self.system_prompt:
            messages = [ChatMessage.from_system(self.system_prompt), message]
        else:
            messages = [message]

        openai_formatted_messages = [_convert_message_to_openai_format(msg) for msg in messages]
        completion: ChatCompletion = self.client.chat.completions.create(
            model=self.model,
            messages=openai_formatted_messages,
            **generation_kwargs
        )
        completions = [self._build_structured_message(completion, choice) for choice in completion.choices]
        for response in completions:
            self._check_finish_reason(response)

        return {
            "replies": [message.content for message in completions],
            "meta": [message.meta for message in completions],
        }

    def _build_structured_message(self, completion: Any, choice: Any) -> ChatMessage:
        chat_message = ChatMessage.from_assistant(choice.message.content or "")
        chat_message.meta.update(
            {
                "model": completion.model,
                "index": choice.index,
                "finish_reason": choice.finish_reason,
                "usage": dict(completion.usage),
            }
        )
        return chat_message


class Metadata(BaseModel):
    PrimaryQuestion: str = Field(description="De belangrijkste vraag die dit item beantwoordt.")
    PrimaryTheme: str = Field(description="Het hoofdthema waar dit item bij hoort.")
    SecondaryThemes: List[str] = Field(description="Specifieke subthema's die het hoofdthema verfijnen.")
    Entities: List[str] = Field(description="Belangrijke personen, plaatsen of objecten die aan het item zijn gekoppeld.")
    EntityRelationships: List[str] = Field(description="Beschrijvingen van relaties tussen entiteiten.")
    TimePeriod: str = Field(description="De historische periode of datum die met het item wordt geassocieerd.")
    Location: str = Field(description="De geografische context die bij het item hoort.")
    AssetType: str = Field(description="Het type item (bijv. Brief, Foto, Kaart).")
    StorylineDimension: str = Field(description="Dominante narratieve structuur (Chronologisch, Entiteitgericht, Emotiegedreven).")
    NarrativeFocus: str = Field(description="Hoe het item bijdraagt aan het verhaal of narratief.")
    Keywords: List[str] = Field(description="Extra zoekwoorden of tags voor betere zoekresultaten.")
    ExplorationTags: List[str] = Field(description="Gerelateerde thema's of onderwerpen voor verdere verkenning.")
    FollowUpQuestionTags: List[str] = Field(description="Tags voor het genereren van vervolgvragen.")
    Summary: Optional[str] = Field(description="Een korte samenvatting van de volledige tekst.", default=None)
    Sender: Optional[str] = Field(description="Naam van de afzender.", default=None)
    Recipient: Optional[str] = Field(description="Naam van de ontvanger.", default=None)
    DateSent: Optional[str] = Field(description="Datum waarop de brief is verstuurd.", default=None)
    LetterType: Optional[str] = Field(description="Type brief (bijv. Persoonlijk, Officieel).", default=None)
    ContentSummary: Optional[str] = Field(description="Korte samenvatting van de inhoud van de brief.", default=None)
    Scale: Optional[str] = Field(description="Schaal van de kaart (bijv. 1:5000).", default=None)
    MapFeatures: Optional[str] = Field(description="Opvallende kenmerken, zoals gebouwen, grenzen, rivieren.", default=None)
    DateCreated: Optional[str] = Field(description="Datum waarop de kaart is gemaakt.", default=None)
    LocationCovered: Optional[str] = Field(description="Gebieden of plaatsen die op de kaart worden weergegeven.", default=None)
    Photographer: Optional[str] = Field(description="Naam van de fotograaf.", default=None)
    DateTaken: Optional[str] = Field(description="Datum waarop de foto is genomen.", default=None)
    Event: Optional[str] = Field(description="Gebeurtenis die op de foto is vastgelegd.", default=None)
    PeopleInPhoto: Optional[str] = Field(description="Namen van personen op de foto.", default=None)
    ArticleTopic: Optional[str] = Field(description="Onderwerp of thema van het artikel.", default=None)
    Author: Optional[str] = Field(description="Naam van de auteur.", default=None)
    PublicationDate: Optional[str] = Field(description="Datum van publicatie.", default=None)
    Source: Optional[str] = Field(description="Naam van het tijdschrift of de bron.", default=None)


@component
class OpenAIGenerator(BaseOpenAIGenerator):
    @component.output_types(replies=List[str], meta=List[Dict[str, Any]], structured_reply=BaseModel)
    def run(
        self,
        prompt: str,
        generation_kwargs: Optional[Dict[str, Any]] = None
    ):
        generation_kwargs = {**self.generation_kwargs, **(generation_kwargs or {})}
        if "response_format" in generation_kwargs.keys():
            system_prompt = ChatMessage.from_system(ANTHROPIC_DEFAULT_PROMPT)
            message = ChatMessage.from_user(prompt)
            messages = [system_prompt, message]
            openai_formatted_messages = [_convert_message_to_openai_format(msg) for msg in messages]
            completion: ChatCompletion = self.client.chat.completions.create(
                model="gpt-4o",
                messages=openai_formatted_messages,
                **generation_kwargs
            )
            completions = [self._build_structured_message(completion, choice) for choice in completion.choices]
            for response in completions:
                self._check_finish_reason(response)
            return {
                "replies": [message.content for message in completions],
                "meta": [message.meta for message in completions],
                "structured_reply": completions[0].content
            }
        else:
            return super().run(prompt, generation_kwargs)

    def _build_structured_message(self, completion: Any, choice: Any) -> ChatMessage:
        chat_message = ChatMessage.from_assistant(choice.message.content or "")
        chat_message.meta.update(
            {
                "model": completion.model,
                "index": choice.index,
                "finish_reason": choice.finish_reason,
                "usage": dict(completion.usage),
            }
        )
        return chat_message


@component
class MetadataEnricher:
    def __init__(self, metadata_model: BaseModel, prompt: str = DEFAULT_PROMPT):
        self.metadata_model = metadata_model
        self.metadata_prompt = prompt
        self.pipeline = Pipeline()
        

    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]):
        documents_with_meta = []
        for document in documents:
            # Extract the metadata, content, and URL from the document
            content = document.content
            url = document.meta.get("link")
            beschrijving = document.meta.get("beschrijving")
            title = document.meta.get("Title")
            
            prompt = f"""
            {self.metadata_prompt}
            URL: {url}
            Titel: {title}
            Beschrijving: {beschrijving}
            Inhoud van het item: {content}
            """
            llm = OpenAIGenerator(generation_kwargs={"response_format": {"type": "json_object"}})
            self.pipeline.add_component(name="llm", instance=llm)
            result = self.pipeline.run(data={"llm": {"prompt": prompt}})
            metadata_str = result['llm']['replies'][0]
            try:
                
                metadata = json.loads(metadata_str)
                document.meta.update(metadata)
                documents_with_meta.append(document)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError: {e} \n\n for reply \n\n {metadata_str}")
                continue
            except ValidationError as e:
                print(f"Pydantic validation error: {e} \n\n for reply \n\n {metadata_str}")
                continue
            except Exception as e:
                 print(f"Unexpected error: {e} \n\n for reply \n\n {metadata_str}")
                 continue
            self.pipeline.remove_component("llm")

        return {"documents": documents_with_meta}

In [9]:
def setup_processing_pipeline():
    pipeline = Pipeline()
    metadata_enricher = MetadataEnricher(metadata_model=Metadata)
    
    pipeline.add_component("metadata_enricher", metadata_enricher)
    pipeline.add_component("cleaner", DocumentCleaner())
    pipeline.add_component("translator", DocumentTranslator())
    pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=15, split_overlap=3))
    
    pipeline.connect("cleaner", "splitter")
    pipeline.connect("splitter", "translator")
    pipeline.connect("translator", "metadata_enricher")
    
    
    return pipeline

In [10]:
pipeline = setup_processing_pipeline()

result = pipeline.run(data={
    "cleaner": {"documents": [docs[9]]}},
    include_outputs_from=["splitter", "translator"]
)

INFO:haystack.core.pipeline.pipeline:Running component cleaner


TypeError: DocumentCleaner expects a List of Documents as input.

In [16]:
result.get('translator')

{'docs': [Document(id=bfdd75b39d687ea87b80841b839e5296c08403953aee8a1db68d6f4525e52b9a, content: 'Geachte brief overgeleverd, ik zal hem helpen zo snel als daar gelegenheid voor is, maar de koning h...', meta: {'ID': 722015, 'AET_ID': 4, 'num_scans': 204, 'invnr': 1001.2735, 'GUID': '609C5B9947C44642E0534701000A17FD', 'beschrijving': 'Brieven gericht aan Godard Adriaan van Reede afkomstig van zijn zoon Godard van Reede-Ginkel, met minuten van aan deze verzonden brieven, 1667-1691 2735.  Brieven, 1690-1691', 'link': 'https://hetutrechtsarchief.nl/collectie/609C5B9947C44642E0534701000A17FD', 'representatieve afbeelding': 'https://proxy.archieven.nl/large/39/609C5C2FDE734642E0534701000A17FD', 'Context': '1001 Huis Amerongen  /  Inventaris / 2. Stukken betreffende heren en vrouwen van Amerongen en Zuilenstein en hun bloedverwanten / 2.1. Familie van Reede / 2.1.07. Godard Adriaan van Reede x Margareta Turnor / 2.1.07.1. Persoonlijk / Brieven gericht aan Godard Adriaan van Reede afkomstig v

In [38]:
translated = result.get('translator').get('docs')

In [40]:
test_pipeline = Pipeline()

formatted_docs = test_pipeline.run(data={"metadata_enricher": {"documents": translated}}, include_outputs_from=["metadata_enricher"])

INFO:haystack.core.pipeline.pipeline:Running component metadata_enricher
INFO:haystack.core.pipeline.pipeline:Running component llm


Prompt is 
            
**Instructies voor het LLM:**

Je bent een expert in het extraheren van metadata uit archiefdocumenten. Je output moet een JSON object zijn dat alle metadata bevat. Alle tekstuele metadata moet in het Nederlands zijn.

**Invoer:**
- **URL:** {{url}}
- **Titel:** {{title}}
- **Beschrijving:** {{beschrijving}}
- **Inhoud:** {{document}}

**Doel:** Genereer een JSON-object met de volgende metadata, afgeleid uit de input:

```json{    "AssetID": "unieke identificatiecode",    "Title": "officiële titel",    "Description": "korte samenvatting van de inhoud",    "PrimaryQuestion": "belangrijkste vraag die het item beantwoordt",    "PrimaryTheme": "hoofdthema van het item",    "SecondaryThemes": ["specifiek subthema 1", "specifiek subthema 2"],    "Entities": ["belangrijke persoon 1", "belangrijke plaats 1", "belangrijk object 1"],    "EntityRelationships": ["beschrijving van relatie 1", "beschrijving van relatie 2"],    "TimePeriod": "historische periode of datum",    

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:haystack.core.pipeline.pipeline:Running component llm


Completions: [ChatMessage(content='\n{\n    "AssetID": "609C5B9947B64642E0534701000A17FD",\n    "Title": "Brieven gericht aan Godard Adriaan van Reede",\n    "Description": "Een verzameling brieven van Margareta Turnor aan haar echtgenoot Godard Adriaan van Reede, gedateerd 1667 juni-augustus en 1671 september-1672 december.",\n    "PrimaryQuestion": "Hoe communiceerde Margareta Turnor met Godard Adriaan van Reede in deze periode?",\n    "PrimaryTheme": "Familiecorrespondentie in de 17e eeuw",\n    "SecondaryThemes": [\n        "Politieke spanningen in de 17e eeuw",\n        "Persoonlijke betrekkingen"\n    ],\n    "Entities": [\n        "Godard Adriaan van Reede",\n        "Margareta Turnor",\n        "heer Rodeburch",\n        "heer Luchtere"\n    ],\n    "EntityRelationships": [\n        "Margareta Turnor is de echtgenote van Godard Adriaan van Reede",\n        "Politieke connecties tussen de heer Luchtere en heer Rodeburch"\n    ],\n    "TimePeriod": "1667",\n    "Location": "Midda

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:haystack.core.pipeline.pipeline:Running component llm


...

Completions: [ChatMessage(content='\n{\n    "AssetID": "609C5B9947B84642E0534701000A17FD",\n    "Title": "Brieven aan Godard Adriaan van Reede van zijn echtgenote Margareta Turnor",\n    "Description": "Brieven uit de periode 1667-1691 van Margareta Turnor aan Godard Adriaan van Reede, waarin zij spreken over bouwprojecten en persoonlijke zaken.",\n    "PrimaryQuestion": "Hoe verliepen de persoonlijke en zakelijke interacties tussen Margareta Turnor en haar echtgenoot Godard Adriaan van Reede?",\n    "PrimaryTheme": "Persoonlijke correspondentie",\n    "SecondaryThemes": ["Bouwprojecten", "Familiebanden"],\n    "Entities": ["Margareta Turnor", "Godard Adriaan van Reede", "Rietvelt"],\n    "EntityRelationships": ["Margareta Turnor als de echtgenote van Godard Adriaan van Reede", "Rietvelt als projectmanager voor bouwwerkzaamheden"],\n    "TimePeriod": "1676 mei-sept., 1677 maart-juni",\n    "Location": "Nederlands context, specifiek rond Amsterdam en Amerongen",\n    "AssetType": "Brief

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:haystack.core.pipeline.pipeline:Running component llm


Completions: [ChatMessage(content='{\n    "AssetID": "609C5B9947B84642E0534701000A17FD",\n    "Title": "Brieven van Margareta Turnor aan Godard Adriaan van Reede",\n    "Description": "Een verzameling brieven van Margareta Turnor aan haar echtgenoot Godard Adriaan van Reede, waarin zij verschillende persoonlijke en zakelijke onderwerpen bespreken.",\n    "PrimaryQuestion": "Hoe verliep de communicatie tussen Margareta Turnor en Godard Adriaan van Reede?",\n    "PrimaryTheme": "Communicatie binnen een adellijke familie",\n    "SecondaryThemes": [\n        "Financieel beheer",\n        "Familieverhoudingen"\n    ],\n    "Entities": [\n        "Margareta Turnor",\n        "Godard Adriaan van Reede",\n        "Rietvelt"\n    ],\n    "EntityRelationships": [\n        "Margareta Turnor is de echtgenote van Godard Adriaan van Reede",\n        "Rietvelt is een werknemer betrokken bij het bouwproject"\n    ],\n    "TimePeriod": "1676 mei-september, 1677 maart-april",\n    "Location": "Amerongen

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Completions: [ChatMessage(content='{\n    "AssetID": "609C5B9947B84642E0534701000A17FD",\n    "Title": "Brieven gericht aan Godard Adriaan van Reede en geschreven door Margareta Turnor",\n    "Description": "Een verzameling brieven geschreven door Margareta Turnor aan haar echtgenoot Godard Adriaan van Reede, waarin zij verslag doet van verschillende zaken en persoonlijke ervaringen gedurende de jaren 1676-1677.",\n    "PrimaryQuestion": "Hoe beschrijft Margareta Turnor de situatie en gebeurtenissen tijdens de oorlog en haar persoonlijke leven in de brieven aan haar man?",\n    "PrimaryTheme": "Persoonlijke correspondentie tijdens oorlogstijd",\n    "SecondaryThemes": [\n        "Familieleven in de 17e eeuw",\n        "Militaire en politieke situaties",\n        "Bouw en huishouding"\n    ],\n    "Entities": [\n        "Margareta Turnor",\n        "Godard Adriaan van Reede",\n        "Sas van Gent",\n        "Graaf Tromp",\n        "Keurvorst van Brandenburgh",\n        "Rietvelt",\n  

In [57]:
formatted_docs.get("metadata_enricher").get("documents")[9].meta.get("FullText")

'volledige tekst zoals geleverd in de inhoud'